In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
train.head()

In [ ]:
output = train['label'].astype('float32')
# Xtr=Xtr.drop(columns=['label'], inplace=True)
# output.head()


In [ ]:
import seaborn as sns
sns.countplot(x='label', data=train)

In [ ]:
import tensorflow as tf
test = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
test.head()

In [ ]:
y_train = train['label'].astype('float32')
X_train = train.drop(['label'], axis=1).astype('int32')
X_test = test.astype('float32')
X_train.shape, y_train.shape, X_test.shape

In [ ]:
X_train = X_train/255
X_test = X_test/255


In [ ]:
X_train = X_train.values.reshape([-1, 28, 28, 1])
X_test =X_test.values.reshape([-1, 28, 28, 1])


In [ ]:
from keras.utils.np_utils import to_categorical
y_train = to_categorical(y_train, num_classes = 10)
y_train.shape

Now plot the image

In [ ]:
from sklearn.model_selection import train_test_split
x, xt, y, yt = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_train[9][:,:,0])
plt.title(y_train[9].argmax());


In [4]:
from keras.layers import Input,InputLayer, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Sequential,Model
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
import keras
from keras import backend as K

In [5]:
inputsize = [28, 28, 1]
xinp = Input(inputsize)


x = Conv2D(64,(3,3),strides=(1,1),name='layer_conv1',padding='same')(xinp)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool1')(x)
# layer 2
x = Conv2D(32,(3,3),strides=(1,1),name='layer_conv2',padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool2')(x)
# layer 3
x = Conv2D(16,(3,3),strides=(1,1),name='layer_conv3',padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool3')(x)
# layer 4
x = Conv2D(8,(3,3),strides=(1,1),name='layer_conv4',padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D((2,2),name='maxPool4')(x)
# fc
x = Flatten()(x)
x = Dense(64,activation ='relu',name='fc0')(x)
x = Dropout(0.25)(x)
x = Dense(32,activation ='relu',name='fc1')(x)
x = Dropout(0.25)(x)
x = Dense(10,activation ='softmax',name='fc2')(x)

conv_model = Model(inputs=xinp, outputs=x, name='Predict')
conv_model.summary()

Model: "Predict"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 layer_conv1 (Conv2D)        (None, 28, 28, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 28, 28, 64)        0         
                                                                 
 maxPool1 (MaxPooling2D)     (None, 14, 14, 64)        0         
                                                                 
 layer_conv2 (Conv2D)        (None, 14, 14, 32)        18464     
                                                           

In [6]:
# sequential model
model_seq = Sequential()
model_seq.add(Conv2D(64,(3,3),strides=(1,1),name='layer_conv1',padding='same',input_shape=inputsize))
model_seq.add(BatchNormalization())
model_seq.add(Activation('relu'))
model_seq.add(MaxPooling2D((2,2),name='maxPool1'))
# layer 2
model_seq.add(Conv2D(32,(3,3),strides=(1,1),name='layer_conv2',padding='same'))
model_seq.add(BatchNormalization())
model_seq.add(Activation('relu'))
model_seq.add(MaxPooling2D((2,2),name='maxPool2'))
# fc
model_seq.add(Flatten())
model_seq.add(Dense(64,activation ='relu',name='fc0'))
model_seq.add(Dropout(0.25))
model_seq.add(Dense(32,activation ='relu',name='fc1'))
model_seq.add(Dropout(0.25))
model_seq.add(Dense(10,activation ='softmax',name='fc2'))
model_seq.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_conv1 (Conv2D)        (None, 28, 28, 64)        640       
                                                                 
 batch_normalization_4 (Batc  (None, 28, 28, 64)       256       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 28, 28, 64)        0         
                                                                 
 maxPool1 (MaxPooling2D)     (None, 14, 14, 64)        0         
                                                                 
 layer_conv2 (Conv2D)        (None, 14, 14, 32)        18464     
                                                                 
 batch_normalization_5 (Batc  (None, 14, 14, 32)       128       
 hNormalization)                                        

In [ ]:
conv_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
conv_model.fit(X_train, y_train, batch_size=100, epochs=20)

In [ ]:
y_pred = conv_model.predict(X_test)
y_pred = np.argmax(y_pred,axis=1)
my_submission = pd.DataFrame({'ImageId': list(range(1, len(y_pred)+1)), 'Label': y_pred})
my_submission.to_csv('submission.csv', index=False)

In [1]:
from tensorflow.keras import layers
# import alexnet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard


NameError: name 'Input' is not defined